In [30]:
! pip install ipykernel

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import pandas as pd
import os

In [10]:

import pandas as pd
from langchain_core.documents import Document


def dataconveter():
    product_data=pd.read_csv("/Users/sachinsen/Documents/Generative Ai/Projects/EComchatBot/data/flipkart_product_review.csv")

    data=product_data[["product_title","review"]]

    product_list = []

    # Iterate over the rows of the DataFrame
    for index, row in data.iterrows():
        # Construct an object with 'product_name' and 'review' attributes
        obj = {
                'product_name': row['product_title'],
                'review': row['review']
            }
        # Append the object to the list
        product_list.append(obj)

        
            
    docs = []
    for entry in product_list:
        metadata = {"product_name": entry['product_name']}
        doc = Document(page_content=entry['review'], metadata=metadata)
        docs.append(doc)
    return docs

In [11]:
from langchain_astradb import AstraDBVectorStore
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
#from dotenv import load_dotenv
import os
import pandas as pd

In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


embedding = download_hugging_face_embeddings()

In [13]:
ASTRA_DB_API_ENDPOINT = "https://f06a2175-95ec-423e-925f-c71d6830b378-us-east-2.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:FoXdnikeOwRGxyhYTtWpPcIE:d98217b3d29e8599a0a50ff3e6ba213152ee4a6f44172f4fec82aaece66177a1"
ASTRA_DB_KEYSPACE = "default_keyspace"

In [15]:
vstore =  AstraDBVectorStore(
                embedding=embedding,
                collection_name="ecomchatbot",
                api_endpoint=ASTRA_DB_API_ENDPOINT,
                token=ASTRA_DB_APPLICATION_TOKEN,
                namespace=ASTRA_DB_KEYSPACE
            )

In [23]:
#from ecomchatbot.Stage01_dataconverter import dataconverter

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


embedding = download_hugging_face_embeddings()

def data_ingestion(status):
   vstore =  AstraDBVectorStore(
                embedding=embedding,
                collection_name="ecomchatbot",
                api_endpoint=ASTRA_DB_API_ENDPOINT,
                token=ASTRA_DB_APPLICATION_TOKEN,
                namespace=ASTRA_DB_KEYSPACE
            )
   
   storage = status

   if storage == None:
       docs = dataconveter()
       inserted_ids = vstore.add_documents(docs)
       
   else:
        return vstore
   
   return vstore, inserted_ids

/Users/sachinsen/Documents/Generative Ai/Projects/EComchatBot/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
from IPython.display import Markdown , display

In [30]:
vstore,inserted_ids=data_ingestion(None)
print(f"\nInserted {len(inserted_ids)} documents.")
results = vstore.similarity_search("what is name of headphone under 5000")
for res in results:
    Markdown(print(f"* {res.page_content} [{res.metadata}]"))


Inserted 450 documents.
* Lovely colour and premium looking and connectivity is awesome and impressive better than 3k-4k rs headphones    Trust without and double thought go for it [{'product_name': 'realme Buds Wireless Bluetooth Headset'}]
* Lovely colour and premium looking and connectivity is awesome and impressive better than 3k-4k rs headphones    Trust without and double thought go for it [{'product_name': 'realme Buds Wireless Bluetooth Headset'}]
* This is a wonderful product Unless you have received a faulty unit. Let me help you here Pros -Best in class buildBest in class features (quick pairing and switching, super quick charge)Best in class battery lifeMost comfortable earphone (i have tried nearly all good earphones under 3000 both wired and wireless. Comfort of these are superb)Cons -Sound is like a typical 1000rs earphone.Bass is there but not that much.But still i am rating it 5* why because my friend No OTHER wirele... [{'product_name': 'OnePlus Bullets Wireless Z Bl